In [1]:
!java -version

openjdk version "1.8.0_362-362"
OpenJDK Runtime Environment (build 1.8.0_362-362-b09)
OpenJDK 64-Bit Server VM (build 25.362-b09, mixed mode)


# **IRIS Naive Bayes Classification with Pyspark**

In [2]:
# install
# !pip install Pyspark
!pip install pyspark==3.3.1

# Conexión con el cluster de Spark con pyspark

In [3]:
import os

# Obtener todas las variables de entorno
env_variables = os.environ

# Imprimir todas las variables de entorno de forma legible
for key, value in env_variables.items():
    if "AWS" in key:
        print(f"{key}: {value}")


AWS_REGION: us-east-1
AWS_SECRET_ACCESS_KEY: t4bl4red0nd4
AWS_ACCESS_KEY_ID: admin


In [4]:
from pyspark.sql import SparkSession

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4 pyspark-shell'
dst_lh_url  = os.environ.get('DST_LH_URL', 'spark://spark-master-0.spark-headless.demml.svc.cluster.local:7077')
dst_lh_appn = os.environ.get('DST_LH_APPN', 'iris_naive_bayes_classification_pyspark_EDU')
import subprocess
my_pod_ip = subprocess.run(['hostname', '-I'], stdout=subprocess.PIPE).stdout.decode('utf-8').strip(' \n\t')
my_pod_ip

spark = SparkSession.builder \
    .master(dst_lh_url) \
    .appName(dst_lh_appn) \
    .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257,software.amazon.awssdk:s3:2.17.257,software.amazon.awssdk:iam:2.17.257,org.apache.hadoop:hadoop-aws:3.2.3') \
    .config('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config('spark.sql.catalog.spark_catalog','org.apache.iceberg.spark.SparkSessionCatalog') \
    .config('spark.sql.catalog.spark_catalog.catalog-impl','org.apache.iceberg.jdbc.JdbcCatalog') \
    .config('spark.sql.catalog.spark_catalog.uri','jdbc:postgresql://postgresql:5432/iceberg') \
    .config('spark.driver.host', my_pod_ip) \
    .config('spark.sql.catalog.spark_catalog.jdbc.useSSL','false') \
    .config('spark.sql.catalog.spark_catalog.jdbc.user','iceberg') \
    .config('spark.sql.catalog.spark_catalog.jdbc.password','iceberg') \
    .config('spark.sql.catalog.spark_catalog.warehouse','s3a://warehouse/') \
    .config('spark.sql.catalog.spark_catalog.s3.endpoint','http://minio:9000') \
    .config('spark.sql.catalog.spark_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO') \
    .config('spark.sql.defaultCatalog','spark_catalog') \
    .config('spark.hadoop.fs.s3a.endpoint','http://minio:9000') \
    .config('spark.hadoop.fs.s3a.access.key','admin') \
    .config('spark.hadoop.fs.s3a.secret.key','t4bl4red0nd4') \
    .config('spark.hadoop.fs.s3a.path.style.access','true') \
    .config('spark.hadoop.fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()

import datetime as dt
ini = dt.datetime.now()

spark

:: loading settings :: url = jar:file:/home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/coder/.ivy2/cache
The jars for the packages stored in: /home/coder/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.mariadb.jdbc#mariadb-java-client added as a dependency
org.postgresql#postgresql added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
software.amazon.awssdk#s3 added as a dependency
software.amazon.awssdk#iam added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b47cad86-98c7-4b8d-8af6-c1bfba8d7bb5;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.1.0 in central
	found org.mariadb.jdbc#mariadb-java-client;2.3.0 in central
	found org.postgresql#postgresql;42.5.4 in central
	found org.checkerframework#checker-qual;3.5.0 in central
	found software.amazon.awssdk#bundle;2.17.257 in central
	found software.am

23/09/19 07:24:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
iris = spark.table("spark_catalog.default.IRIS_EDU_V5")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [6]:
iris.show()

+------------+-----------+------------+-----------+----------+------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|target|
+------------+-----------+------------+-----------+----------+------+
|         6.4|        2.7|         5.3|        1.9| virginica|     2|
|         5.6|        2.5|         3.9|        1.1|versicolor|     1|
|         6.1|        2.8|         4.0|        1.3|versicolor|     1|
|         6.9|        3.1|         5.1|        2.3| virginica|     2|
|         6.1|        2.8|         4.7|        1.2|versicolor|     1|
|         5.4|        3.9|         1.3|        0.4|    setosa|     0|
|         5.9|        3.0|         4.2|        1.5|versicolor|     1|
|         6.9|        3.1|         5.4|        2.1| virginica|     2|
|         5.0|        3.2|         1.2|        0.2|    setosa|     0|
|         5.0|        2.0|         3.5|        1.0|versicolor|     1|
|         6.4|        2.7|         5.3|        1.9| virginica|     2|
|         5.5|      

In [7]:
iris.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)
 |-- target: integer (nullable = true)



In [8]:
iris.columns

['sepal_length',
 'sepal_width',
 'petal_length',
 'petal_width',
 'variety',
 'target']

In [9]:
from pyspark.ml.feature import VectorAssembler 

featureassembler = VectorAssembler(inputCols=[
    'sepal_length',
    'sepal_width',
    'petal_length',
    'petal_width'
], outputCol='Features')

In [10]:
output = featureassembler.transform(iris)

In [11]:
output.show()

+------------+-----------+------------+-----------+----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|target|         Features|
+------------+-----------+------------+-----------+----------+------+-----------------+
|         6.4|        2.7|         5.3|        1.9| virginica|     2|[6.4,2.7,5.3,1.9]|
|         5.6|        2.5|         3.9|        1.1|versicolor|     1|[5.6,2.5,3.9,1.1]|
|         6.1|        2.8|         4.0|        1.3|versicolor|     1|[6.1,2.8,4.0,1.3]|
|         6.9|        3.1|         5.1|        2.3| virginica|     2|[6.9,3.1,5.1,2.3]|
|         6.1|        2.8|         4.7|        1.2|versicolor|     1|[6.1,2.8,4.7,1.2]|
|         5.4|        3.9|         1.3|        0.4|    setosa|     0|[5.4,3.9,1.3,0.4]|
|         5.9|        3.0|         4.2|        1.5|versicolor|     1|[5.9,3.0,4.2,1.5]|
|         6.9|        3.1|         5.4|        2.1| virginica|     2|[6.9,3.1,5.4,2.1]|
|         5.0|        3.2|      

In [12]:
modeldata = output.select('Features','target')

In [13]:
modeldata.show()

+-----------------+------+
|         Features|target|
+-----------------+------+
|[6.0,2.2,4.0,1.0]|     1|
|[6.2,2.8,4.8,1.8]|     2|
|[5.1,3.8,1.5,0.3]|     0|
|[6.7,3.0,5.2,2.3]|     2|
|[5.1,3.5,1.4,0.2]|     0|
|[5.1,3.3,1.7,0.5]|     0|
|[6.9,3.2,5.7,2.3]|     2|
|[5.1,3.8,1.5,0.3]|     0|
|[6.3,2.9,5.6,1.8]|     2|
|[5.0,2.3,3.3,1.0]|     1|
|[7.2,3.2,6.0,1.8]|     2|
|[6.2,2.2,4.5,1.5]|     1|
|[5.1,3.8,1.6,0.2]|     0|
|[5.7,2.9,4.2,1.3]|     1|
|[4.7,3.2,1.3,0.2]|     0|
|[5.0,2.0,3.5,1.0]|     1|
|[5.7,3.8,1.7,0.3]|     0|
|[5.2,4.1,1.5,0.1]|     0|
|[7.3,2.9,6.3,1.8]|     2|
|[6.7,3.1,5.6,2.4]|     2|
+-----------------+------+
only showing top 20 rows



In [14]:
# split data
train_data, test_data = modeldata.randomSplit([0.8,0.2])

In [15]:
train_data.show()

+-----------------+------+
|         Features|target|
+-----------------+------+
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
+-----------------+------+
only showing top 20 rows



In [16]:
# naive bayes model
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(featuresCol='Features', labelCol='target')

nb = nb.fit(train_data)

In [17]:
# prediction
y_pred = nb.transform(test_data)

In [18]:
y_pred.show()

+-----------------+------+--------------------+--------------------+----------+
|         Features|target|       rawPrediction|         probability|prediction|
+-----------------+------+--------------------+--------------------+----------+
|[4.3,3.0,1.1,0.1]|     0|[-9.8912073406596...|[0.73533777871588...|       0.0|
|[4.3,3.0,1.1,0.1]|     0|[-9.8912073406596...|[0.73533777871588...|       0.0|
|[4.3,3.0,1.1,0.1]|     0|[-9.8912073406596...|[0.73533777871588...|       0.0|
|[4.3,3.0,1.1,0.1]|     0|[-9.8912073406596...|[0.73533777871588...|       0.0|
|[4.3,3.0,1.1,0.1]|     0|[-9.8912073406596...|[0.73533777871588...|       0.0|
|[4.3,3.0,1.1,0.1]|     0|[-9.8912073406596...|[0.73533777871588...|       0.0|
|[4.3,3.0,1.1,0.1]|     0|[-9.8912073406596...|[0.73533777871588...|       0.0|
|[4.3,3.0,1.1,0.1]|     0|[-9.8912073406596...|[0.73533777871588...|       0.0|
|[4.3,3.0,1.1,0.1]|     0|[-9.8912073406596...|[0.73533777871588...|       0.0|
|[4.3,3.0,1.1,0.1]|     0|[-9.8912073406

In [19]:
# confusion matrix
y_pred.groupBy('target', 'prediction').count().show()

+------+----------+-------+
|target|prediction|  count|
+------+----------+-------+
|     2|       1.0|  62092|
|     0|       0.0|1036648|
|     1|       2.0|  82739|
|     2|       2.0| 971770|
|     1|       1.0| 950430|
+------+----------+-------+



In [20]:
# from sklearn.metrics import confusion_matrix
# pred = y_pred.select("prediction").collect()
# orig = y_pred.select("target").collect()
# print(confusion_matrix(orig, pred))

In [21]:
# evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol='target', predictionCol='prediction'
)

evaluator.metricName

Param(parent='MulticlassClassificationEvaluator_4c91ccebfa82', name='metricName', doc='metric name in evaluation (f1|accuracy|weightedPrecision|weightedRecall|weightedTruePositiveRate| weightedFalsePositiveRate|weightedFMeasure|truePositiveRateByLabel| falsePositiveRateByLabel|precisionByLabel|recallByLabel|fMeasureByLabel| logLoss|hammingLoss)')

In [22]:
accuracy = evaluator.evaluate(y_pred)
precision = evaluator.evaluate(y_pred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(y_pred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(y_pred, {evaluator.metricName: 'f1'})

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

Accuracy: 0.9533791303637631
Precision: 0.9534605229883248
Recall: 0.953356660480462
F1 Score: 0.9533551088007102


In [23]:
# close connection to spark
spark.stop()

In [24]:
fin = dt.datetime.now()

print(str(fin - ini))

0:04:28.375295
